In [27]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2 as cv
from numpy.random import seed
seed(42)
import pickle

from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from glob import glob 
%matplotlib inline

In [8]:
IMG_SIZE = 96
IMG_CHANNELS = 3
#TRAIN_SIZE=80000
TRAIN_SIZE = 8000
BATCH_SIZE = 64
EPOCHS = 30


In [9]:
#I used cropped images
train_labels = pd.read_csv('train_labels.csv')
train_labels['label'].value_counts()


0    130908
1     89117
Name: label, dtype: int64

In [10]:
(train_labels.label.value_counts() / len(train_labels)).to_frame()

,label
0,0.594969
1,0.405031


In [11]:
train_neg = train_labels[train_labels['label']==0].sample(TRAIN_SIZE,random_state=45)
train_pos = train_labels[train_labels['label']==1].sample(TRAIN_SIZE,random_state=45)

train_data = pd.concat([train_neg, train_pos], axis=0).reset_index(drop=True)

train_data = shuffle(train_data)


In [12]:
train_data['label'].value_counts()

1    8000
0    8000
Name: label, dtype: int64

In [13]:
def append_ext(fn):
    return fn+".tif"

In [14]:
y = train_data['label']
train_df, valid_df = train_test_split(train_data, test_size=0.2, random_state=45, stratify=y)

print(train_df.shape)
print(valid_df.shape)

(12800, 2)
(3200, 2)


In [15]:
train_df['id'] = train_df['id'].apply(append_ext)
valid_df['id'] = valid_df['id'].apply(append_ext)
train_df.head()

,id,label
12571,4e2c9a6d0be94031e10013cffc9dd8747520e9bd.tif,1
875,f87538389122888cd4c56cbe1b9b22b79a2a1c93.tif,0
4506,43c463fdc93bc0ad76bb64b5a5b3761648b70c2d.tif,0
12453,816336cfb3623fa6693bf0a0012b897dddec6b7f.tif,1
2101,72da772f3577ecc920101422201d0d3f03e87c07.tif,0


In [16]:
train_datagen = ImageDataGenerator(rescale=1/255)
valid_datagen = ImageDataGenerator(rescale=1/255)

In [17]:
BATCH_SIZE = 64
train_path = './train'
train_df['label'] = train_df['label'].astype(str)
valid_df['label'] = valid_df['label'].astype(str)

train_loader = train_datagen.flow_from_dataframe(
    dataframe = train_df,
    directory = train_path,
    x_col = 'id',
    y_col = 'label',
    batch_size = BATCH_SIZE,
    seed = 42,
    shuffle = True,
    class_mode = 'categorical',
    target_size = (32,32)
)

valid_loader = valid_datagen.flow_from_dataframe(
    dataframe = valid_df,
    directory = train_path,
    x_col = 'id',
    y_col = 'label',
    batch_size = BATCH_SIZE,
    seed = 42,
    shuffle = True,
    class_mode = 'categorical',
    target_size = (32,32)
)


Found 12800 validated image filenames belonging to 2 classes.
Found 3200 validated image filenames belonging to 2 classes.


In [18]:
TR_STEPS = len(train_loader)
VA_STEPS = len(valid_loader)

print(TR_STEPS)
print(VA_STEPS)

200
50


In [20]:
cnn = Sequential([
    Conv2D(32, (3,3), activation = 'relu', padding = 'same', input_shape=(32,32,3)),
    Conv2D(32, (3,3), activation = 'relu', padding = 'same'),
    Conv2D(32, (3,3), activation = 'relu', padding = 'same'),
    MaxPooling2D(2,2),
    Dropout(0.4),
    BatchNormalization(),

    Conv2D(64, (3,3), activation = 'relu', padding = 'same'),
    Conv2D(64, (3,3), activation = 'relu', padding = 'same'),
    Conv2D(64, (3,3), activation = 'relu', padding = 'same'),
    MaxPooling2D(2,2),
    Dropout(0.5),
    BatchNormalization(),
    
    Conv2D(128, (3,3), activation = 'relu', padding = 'same'),
    Conv2D(128, (3,3), activation = 'relu', padding = 'same'),
    Conv2D(128, (3,3), activation = 'relu', padding = 'same'),
    MaxPooling2D(2,2),
    Dropout(0.6),
    BatchNormalization(),

    Flatten(),
     Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(16, activation='relu'),
    Dropout(0.4),
    Dense(8, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(2, activation='softmax')
])


In [21]:
opt = tf.keras.optimizers.Adam(0.001)
cnn.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.AUC()])

In [22]:
h1 = cnn.fit(
    x = train_loader, 
    steps_per_epoch = TR_STEPS, 
    epochs = 30,
    validation_data = valid_loader, 
    validation_steps = VA_STEPS, 
    verbose = 1
)

Epoch 1/30
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2022-03-04 10:56:48.406332: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-03-04 10:56:48.406725: W tensorflow/core/platform/profile_utils/cpu_utils.cc:126] Failed to get CPU frequency: 0 Hz


200/200 [==============================] - ETA: 0s - loss: 0.7263 - accuracy: 0.5155 - auc: 0.5168WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x177d32c10> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
200/200 [==============================] - 14s 66ms/step - loss: 0.7262 - accuracy: 0.5156 - auc: 0.5169 - val_loss: 29.27

In [23]:
tf.keras.backend.set_value(cnn.optimizer.learning_rate, 0.0001)

In [24]:
h2 = cnn.fit(
    x = train_loader, 
    steps_per_epoch = TR_STEPS, 
    epochs = 30,
    validation_data = valid_loader, 
    validation_steps = VA_STEPS, 
    verbose = 1
)

Epoch 1/30
200/200 [==============================] - 13s 63ms/step - loss: 0.5673 - accuracy: 0.7268 - auc: 0.7884 - val_loss: 0.5287 - val_accuracy: 0.7584 - val_auc: 0.8158
Epoch 2/30
200/200 [==============================] - 12s 61ms/step - loss: 0.5660 - accuracy: 0.7273 - auc: 0.7897 - val_loss: 0.5266 - val_accuracy: 0.7606 - val_auc: 0.8176
Epoch 3/30
200/200 [==============================] - 12s 61ms/step - loss: 0.5665 - accuracy: 0.7266 - auc: 0.7883 - val_loss: 0.5267 - val_accuracy: 0.7588 - val_auc: 0.8182
Epoch 4/30
200/200 [==============================] - 12s 60ms/step - loss: 0.5706 - accuracy: 0.7216 - auc: 0.7844 - val_loss: 0.5279 - val_accuracy: 0.7606 - val_auc: 0.8183
Epoch 5/30
200/200 [==============================] - 12s 60ms/step - loss: 0.5653 - accuracy: 0.7279 - auc: 0.7913 - val_loss: 0.5271 - val_accuracy: 0.7609 - val_auc: 0.8186
Epoch 6/30
200/200 [==============================] - 12s 62ms/step - loss: 0.5640 - accuracy: 0.7252 - auc: 0.7909 - va

In [25]:
tf.keras.backend.set_value(cnn.optimizer.learning_rate, 0.00001)

In [26]:
 h3 = cnn.fit(
    x = train_loader, 
    steps_per_epoch = TR_STEPS, 
    epochs = 30,
    validation_data = valid_loader, 
    validation_steps = VA_STEPS, 
    verbose = 1
)

Epoch 1/30
200/200 [==============================] - 13s 65ms/step - loss: 0.5631 - accuracy: 0.7291 - auc: 0.7931 - val_loss: 0.5277 - val_accuracy: 0.7606 - val_auc: 0.8177
Epoch 2/30
200/200 [==============================] - 13s 67ms/step - loss: 0.5630 - accuracy: 0.7253 - auc: 0.7933 - val_loss: 0.5284 - val_accuracy: 0.7606 - val_auc: 0.8177
Epoch 3/30
200/200 [==============================] - 13s 65ms/step - loss: 0.5659 - accuracy: 0.7267 - auc: 0.7901 - val_loss: 0.5281 - val_accuracy: 0.7613 - val_auc: 0.8177
Epoch 4/30
200/200 [==============================] - 13s 63ms/step - loss: 0.5632 - accuracy: 0.7255 - auc: 0.7924 - val_loss: 0.5285 - val_accuracy: 0.7594 - val_auc: 0.8172
Epoch 5/30
200/200 [==============================] - 13s 65ms/step - loss: 0.5621 - accuracy: 0.7302 - auc: 0.7933 - val_loss: 0.5283 - val_accuracy: 0.7619 - val_auc: 0.8174
Epoch 6/30
200/200 [==============================] - 13s 65ms/step - loss: 0.5623 - accuracy: 0.7306 - auc: 0.7942 - va